In [128]:
import pandas as pd
import scipy.stats
import numpy as np
import os

In [129]:
data = os.listdir("processed data")

In [130]:
ns_files = [a for a in data if "ns" in a ]

In [131]:
past_files = [a for a in data if ("ns" not in a and "line" not in a)]

In [132]:
df = pd.read_csv("processed data/"+ns_files[0])

In [149]:
new_df

pid                       Player  Season    AVG  FPTS_TG  \
0   16413   Patrick Mahomes II KC (10)    2023   15.0      NaN   
1   17298          Josh Allen BUF (13)    2023   20.0      NaN   
2   19275         Jalen Hurts PHI (10)    2023   22.7      NaN   
3   17233       Lamar Jackson BAL (13)    2023   34.3      NaN   
4   19196           Joe Burrow CIN (7)    2023   35.0      NaN   
5   18635       Justin Herbert LAC (5)    2023   46.0      NaN   
6   19781       Justin Fields CHI (13)    2023   47.0      NaN   
7   19780      Trevor Lawrence JAC (9)    2023   56.0      NaN   
8   15600         Dak Prescott DAL (7)    2023   77.7      NaN   
9   16398       Deshaun Watson CLE (5)    2023   82.7      NaN   
10  19198      Tua Tagovailoa MIA (10)    2023   90.3      NaN   
12  11177        Kirk Cousins MIN (13)    2023  107.0      NaN   
11   9001        Aaron Rodgers NYJ (7)    2023  107.0      NaN   
13  18232        Daniel Jones NYG (13)    2023  109.0      NaN   
14  11687           Geno Smith SEA (5)    2023  116.0      NaN   
0   24347  Anthony Richardson IND (11)    2023  116.3      NaN   
16  15501           Jared Goff DET (9)    2023  124.7      NaN   
17  11180       Russell Wilson DEN (9)    2023  146.3      NaN   
0   22900          Bryce Young CAR (7)    2023  152.3      NaN   
19  19797           Brock Purdy SF (9)    2023  155.7      NaN   
20  22722        Kenny Pickett PIT (6)    2023  156.7      NaN   
22  19246           Jordan Love GB (6)    2023  162.7      NaN   
0   23071          C.J. Stroud HOU (7)    2023  162.7      NaN   
23  12092           Derek Carr NO (11)    2023  167.0      NaN   
24  18600        Kyler Murray ARI (14)    2023  171.7      NaN   
25   9451    Matthew Stafford LAR (10)    2023  173.7      NaN   
26  12208      Jimmy Garoppolo LV (13)    2023  179.7      NaN   
27  20156            Mac Jones NE (11)    2023  186.0      NaN   
29  11174       Ryan Tannehill TEN (7)    2023  197.5      NaN   
30  23045          Sam Howell WAS (14)    2023  202.5      NaN   
31  20080      Desmond Ridder ATL (11)    2023  213.5      NaN   
28  20082            Trey Lance SF (9)    2023  217.7      NaN   
35   9702          Colt McCoy ARI (14)    2023  218.0      NaN   
36  17447          Mike White MIA (10)    2023  220.0      NaN   
37  18562  Gardner Minshew II IND (11)    2023  222.0      NaN   
32  17237        Baker Mayfield TB (5)    2023  225.0      NaN   
33  19800            Kyle Trask TB (5)    2023  250.0      NaN   
0   22987           Will Levis TEN (7)    2023  258.5      NaN   
0   23692                  Ryan Willis    2023  294.0      NaN   
39  19220                  Jacob Eason    2023  295.0      NaN   
0   24201                   Drew Plitt    2023  298.0      NaN   

    Y/A_PASSING_avg  FPTS/G_MISC_avg  CMP_PASSING/G_avg  ATT_PASSING/G_avg  \
0          7.833333        24.200000          25.745098          38.674510   
1          7.400000        24.766667          23.367647          35.818627   
2          7.500000        18.166667          14.400000          23.111111   
3          7.233333        21.400000          17.850000          28.022222   
4          7.666667        20.033333          24.621078          36.260784   
5          7.200000        21.100000          26.839216          40.104575   
6          7.000000        15.950000          13.025000          21.850000   
7          6.500000        15.300000          21.941176          34.882353   
8          7.733333        22.100000          25.858333          38.161111   
9          7.700000        19.300000          20.187500          31.166667   
10         7.333333        15.700000          19.584615          29.871795   
12         7.633333        19.066667          23.334559          35.045343   
11         7.566667        20.033333          22.237745          32.648284   
13         6.700000        15.933333          20.301136          31.439394   
14         7.166667        11.133333          14.573529        

In [173]:
for ns, ps in zip(ns_files, past_files):
    
    df = pd.read_csv('processed data/'+ns)
    pdf = pd.read_csv("processed data/"+ps)
    rep_df = df.copy()
    
    pos = ns[:2]
    
    infos = df[df["YDS_RUSHING/G_avg"].isna()][["pid", "AVG"]].values
    
    reps = []
    for info in infos:
        rookie_adp = info[1]
        rookie_pid = info[0]

        rookie_infos = rep_df[rep_df.pid == rookie_pid]
        nearest_values = []
        for i in pdf.Season.value_counts().index.values:
            sdf = pdf[pdf.Season == i]

            adps = sdf.AVG.values

            closest_number = min(adps, key = lambda x: abs( rookie_adp - x))

            closest_player = sdf[sdf.AVG == closest_number].values


            nearest_values.append(closest_player[0][5:-2].tolist())

        rep_values = pd.DataFrame(nearest_values, columns = pdf.columns[5:-2]).mean(numeric_only = True).values.tolist()#.mean(axis=1)

        bdf = rookie_infos.values[0][:5].tolist() + rep_values + rookie_infos.values[0][-3:].tolist()
        reps.append(pd.DataFrame([bdf], columns=df.columns))

    reps_full = pd.concat(reps)
    new_df = pd.concat([rep_df, reps_full])
    new_df = new_df.drop_duplicates(["pid"], keep="last")

    new_df = new_df.sort_values("AVG")
    
    new_df.to_csv(f"processed data/{pos}_rookie.csv", index=False)